<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue May 31 15:21:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

import os 
path_drive = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet-Padel/'

Mounted at /content/gdrive
/content


In [25]:
os.chdir('/content/')
!pwd

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

fatal: destination path 'TrackNet-Padel' already exists and is not an empty directory.
Already up to date.


In [5]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Predict

In [ ]:
for i in range(10):
  video = '/content/gdrive/MyDrive/PadelMatchs/PremierRoma_Semis_LGBC_0_54_'+str(i)+'.mp4'
  !python predict.py --video_path={video} 

2022-05-29 18:45:26.314328: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Number of frames in video and dictionary are not the same!
Fail to load, predict only.
Beginning predicting......
Traceback (most recent call last):
  File "predict.py", line 116, in <module>
    y_pred = model.predict(img_input, batch_size=BATCH_SIZE)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/ops.py", line 4125, in _get_operation_by_name_unsafe
    return self._nodes_by_name[name]
KeyError: 'res_net__track/sequential_10/res_net__bottle_neck_15/batch_normalization_56/FusedBatchNormV3'
2022-05-29 18:45:32.777034: W tensorflow/core/common_runtime/gpu/gpu_bfc_allo

### Test accuracy

In [22]:
file_name = 'Final_PremierQatar_Tie'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean3'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --load_weights={weights_path} # 2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-06-01 06:48:49.152945: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 144
Number of true negative: 64
Number of false positive FP1: 11
Number of false positive FP2: 3
Number of false negative: 166
Accuracy: 0.5360824742268041
Precision: 0.9113924050632911
Recall: 0.4645161290322581
Total Time: 32.25340986251831
(ACC + Pre + Rec)/3: 0.637330336107451
Done......
raw_data/points/Final_PremierQatar_Tie_10.mp4
2022-06-01 06:49:32.997802: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting..

### Train Padel

In [17]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/clean/*_0\.*
raw_data/points/clean/*_1\.*
raw_data/points/clean/*_2\.*
raw_data/points/clean/*_3\.*
raw_data/points/clean/*_4\.*
raw_data/points/clean/*_5\.*
raw_data/points/clean/*_6\.*
mv: cannot stat 'raw_data/points/clean/*_6.*': No such file or directory
raw_data/points/clean/*_7\.*
mv: cannot stat 'raw_data/points/clean/*_7.*': No such file or directory
raw_data/points/clean/*_8\.*
mv: cannot stat 'raw_data/points/clean/*_8.*': No such file or directory


In [ ]:
!rm -rf train_data/*
!python video2img.py 

In [20]:
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-PadelClean3'
!python train.py --save_weights={weights_path}  --batch_size=5 --lr=10  --pre_trained=True  --load_weights={weights_path}

lerning rate: 10.0
2022-05-31 21:48:39.665553: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 1157, Learning rate: 10.0000==========
1157/1157 [==============================] - 468s 393ms/step - loss: 2.8495e-04 - binary_accuracy: 0.9996
==========Epoch 0 start validation==========
Epoch 0 accuracy: 0.575
Epoch 0 precision: 0.885
Epoch 0 recall: 0.544
Epoch 0 average = (accuracy + precision + recall)/3: 0.668
==========Epoch 1, Train steps: 1157, Learning rate: 10.0000==========
1157/1157 [==============================] - 451s 390ms/step - loss: 2.5428e-04 - binary_accuracy: 0.9996
==========Epoch 1 start validation==========
Epoch 1 accuracy: 0.654
Epoch 1 precision: 0.875
Epoch 1 recall: 0.709
Epoch 1 average = (accuracy + precision + recall)/3: 0.746
==========Epoch 2, Train steps: 11